# Analisador de empresas

Tem como objetivo extrair informações relacionadas as empresas de uma determinada região e sua classificação no Google Meu Negócio e Facebook, se houver mídia relacionada.

```sql
CREATE TABLE empresa (
    id             VARCHAR(36)   NOT NULL,
    nome           VARCHAR(255)  NOT NULL,
    atividade      VARCHAR(255),
    pessoaJuridica BOOLEAN,
    emailCliente   VARCHAR(255),
    cep            VARCHAR(20) ,
    cidade         VARCHAR(255),
    estado         VARCHAR(255),
    bairro         VARCHAR(255),
    uf             VARCHAR(5)  ,
    PRIMARY KEY (id)
);

CREATE TABLE empresa_telefone (
    id                VARCHAR(36) NOT NULL,
    id_empresa        VARCHAR(36) NOT NULL,
    ddd               VARCHAR(5),
    telefone          VARCHAR(50) NOT NULL,
    telefoneCompleto  VARCHAR(100),
    whatsapp          BOOLEAN,
    telefone0800      BOOLEAN,
    PRIMARY KEY (id)
);

CREATE TABLE empresa_endereco (
    id         VARCHAR(36)  NOT NULL,
    id_empresa VARCHAR(36)  NOT NULL,
    endereco   VARCHAR(255) NOT NULL,
    PRIMARY KEY (id)
);
```

In [258]:
!pip install requests

In [259]:
!pip install psycopg2-binary

In [260]:
DEBUG = True

def LOG(funcao, mensagem, pulaLinhaAntes = False, pulaLinhaDepois = False):
    if DEBUG == True:
        if pulaLinhaAntes:
            print('\n');
        print('[{}] => {}'.format(funcao, mensagem))
        if pulaLinhaDepois:
            print('\n');

In [261]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import json
import psycopg2
import uuid
import urllib.parse
import requests
import re

In [262]:
def buscaEmpresaNoGoogle(nomeEmpresa):

    LOG('buscaEmpresaNoGoogle', 'Iniciado')

    site = 'https://www.google.com/search?q={}&as_filetype=json'.format(urllib.parse.quote(nomeEmpresa))

    LOG('buscaEmpresaNoGoogle', 'Buscar empresa no google: {}'.format(site))

    resultado = requests.get(url = site)
    bs = BeautifulSoup(resultado.text, 'html.parser')

    LOG('buscaEmpresaNoGoogle', 'Finalizado')

In [263]:
def existeEmpresaComNome(cursor, nome):

    LOG('existeEmpresaComNome', 'Iniciado')

    cursor.execute("SELECT 1 FROM empresa WHERE nome = '{}'".format(nome.replace("'", "''")))
    resultado = cursor.fetchone()
    existeEmpresa = False

    if (resultado != None):
        existeEmpresa = True

    LOG('existeEmpresaComNome', 'Existe empresa? {}'.format(existeEmpresa))
    LOG('existeEmpresaComNome', 'Finalizado')

    return existeEmpresa

In [264]:
def inserirTelefonesEmpresa(conexao, cursor, empresaId, telefones):
    
    LOG('inserirTelefonesEmpresa', 'Iniciado')

    for telefone in telefones:
        
        idTelefone = uuid.uuid4()
        
        LOG('inserirTelefonesEmpresa', 'UUID do registro: {}'.format(str(idTelefone)))
        
        whatsapp = False
        if telefone['whatsapp'] == 'true':
            whatsapp = True
        
        telefone0800 = False
        if telefone['telefone0800'] == 'true':
            telefone0800 = True

        cursor.execute("INSERT INTO empresa_telefone (id, id_empresa, ddd, telefone, telefoneCompleto, whatsapp, telefone0800) VALUES (%s, %s, %s, %s, %s, %s, %s)",(str(idTelefone), str(empresaId), telefone['ddd'], telefone['telefone'], telefone['telefoneCompleto'], whatsapp, telefone0800))
        conexao.commit()

    LOG('inserirTelefonesEmpresa', 'Finalizado')

    return

In [265]:
def inserirEnderecosEmpresa(conexao, cursor, empresaId, enderecos):

    LOG('inserirEnderecosEmpresa', 'Iniciado')

    for endereco in enderecos:
        
        idEndereco = uuid.uuid4()
        
        LOG('inserirEnderecosEmpresa', 'UUID do registro: {}'.format(str(idEndereco)))

        cursor.execute("INSERT INTO empresa_endereco (id, id_empresa, endereco) VALUES (%s, %s, %s)",(str(idEndereco), str(empresaId), endereco))
        conexao.commit()

    LOG('inserirEnderecosEmpresa', 'Finalizado')

    return

In [266]:
def inserirEmpresa(conexao, cursor, empresa):

    LOG('inserirEmpresa', 'Iniciado')

    pessoaJuridica = empresa['pessoaJuridica']
    if pessoaJuridica == False:
        LOG('inserirEmpresa', 'Pessoa fisica: {} nao inserida'.format(empresa['nome']))
        LOG('inserirEmpresa', 'Finalizado')
        return

    if (existeEmpresaComNome(cursor, empresa['nome']) == False):
        
        idEmpresa = uuid.uuid4()
        
        LOG('inserirEmpresa', 'UUID do registro: {}'.format(str(idEmpresa)))
        
        cursor.execute("INSERT INTO empresa (id, nome, atividade, emailCliente, cep, cidade, estado, bairro, uf, pessoaJuridica) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",(str(idEmpresa), empresa['nome'], empresa['atividade'], empresa['emailCliente'], empresa['cep'], empresa['cidade'], empresa['estado'], empresa['bairro'], empresa['uf'], pessoaJuridica))
        conexao.commit()

        inserirTelefonesEmpresa(conexao, cursor, idEmpresa, empresa['telefones'])
        inserirEnderecosEmpresa(conexao, cursor, idEmpresa, empresa['enderecos'])

        LOG('inserirEmpresa', 'Registro inserido: {}'.format(empresa['nome']))

    LOG('inserirEmpresa', 'Finalizado')

    return

In [267]:
def inserirEmpresas(empresas):

    LOG('inserirEmpresas', 'Iniciado')

    nomeBancoDados = 'COLETADEDADOS'
    usuarioBancoDados = 'postgres'
    senhaBancoDados = 'totvs@123'
    hostBancoDados = 'localhost'

    conexao = psycopg2.connect(dbname=nomeBancoDados, user=usuarioBancoDados, password=senhaBancoDados, host=hostBancoDados)
    cursor = conexao.cursor()

    LOG('inserirEmpresas', 'Conexao com banco de dados: {} efetuada'.format(nomeBancoDados))

    for empresa in empresas:
        inserirEmpresa(conexao, cursor, empresa)

    cursor.close()
    conexao.close()

    LOG('inserirEmpresas', 'Conexao com banco de dados: {} finalizada'.format(nomeBancoDados))
    LOG('inserirEmpresas', 'Finalizado')

    return

In [268]:
def buscarEmpresasDaPagina(categoria, cidade, estado, pagina):

    LOG('buscarEmpresasDaPagina', 'Iniciado')

    site = 'https://guiafacil.com/buscaAjax.php?busca={}&cidade={}&tipoBusca=segmento&estado={}&pagina={}'
    site = site.format(categoria, cidade, estado, pagina)

    LOG('buscarEmpresasDaPagina', 'Buscar: {}'.format(site))

    conteudo = urlopen(site).read()
    conteudoJson = json.loads(conteudo)
    resultado = conteudoJson['resultados']

    LOG('buscarEmpresasDaPagina', 'Quantidade de empresas da página: {}'.format(len(resultado)))
    LOG('buscarEmpresasDaPagina', 'Finalizado')

    return resultado

In [271]:
def buscarEmpresas(categoria, cidade, estado):

    LOG('buscarEmpresas', 'Iniciado')

    pagina = 1
    while(True):
        resultadoPagina = buscarEmpresasDaPagina(categoria, cidade, estado, pagina)
        if (len(resultadoPagina) == 0):
            break

        inserirEmpresas(resultadoPagina)
        pagina += 1

    LOG('buscarEmpresas', 'Finalizado')

    return

In [270]:
buscarEmpresas('', 'joinville', 'santa-catarina')

uscar: https://guiafacil.com/buscaAjax.php?busca=&cidade=joinville&tipoBusca=segmento&estado=santa-catarina&pagina=9182
[buscarEmpresasDaPagina] => Quantidade de empresas da página: 0
[buscarEmpresasDaPagina] => Finalizado
[inserirEmpresas] => Iniciado
[inserirEmpresas] => Conexao com banco de dados: COLETADEDADOS efetuada
[inserirEmpresas] => Conexao com banco de dados: COLETADEDADOS finalizada
[inserirEmpresas] => Finalizado
[buscarEmpresasDaPagina] => Iniciado
[buscarEmpresasDaPagina] => Buscar: https://guiafacil.com/buscaAjax.php?busca=&cidade=joinville&tipoBusca=segmento&estado=santa-catarina&pagina=9183
[buscarEmpresasDaPagina] => Quantidade de empresas da página: 0
[buscarEmpresasDaPagina] => Finalizado
[inserirEmpresas] => Iniciado
[inserirEmpresas] => Conexao com banco de dados: COLETADEDADOS efetuada
[inserirEmpresas] => Conexao com banco de dados: COLETADEDADOS finalizada
[inserirEmpresas] => Finalizado
[buscarEmpresasDaPagina] => Iniciado
[buscarEmpresasDaPagina] => Buscar:

JSONDecodeError: Expecting value: line 1 column 1 (char 0)